<a href="https://colab.research.google.com/github/YinterestingProjects/human-wildlife-interactions/blob/main/notebooks/BERTopics_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install pandas==1.4.2 bertopic==0.14.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 82.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pickle
import pandas as pd
from bertopic import BERTopic
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer

RANDOM_SEED = 42

# directory setup
user = "sally"
#user = "jina"
#user = "josh"
     
try:
  if user == "sally":
    directory = '/content/drive/MyDrive/MADS/MADS_Capstone'
  elif user == "jina":
    directory = "JINA: INSERT YOUR PATH HERE"
  elif user == "josh":
    directory = "/content/drive/MyDrive/MADS_Capstone"
except:
  dprint("No user has been selected")

In [4]:
fp = f'{directory}/data/processed/desc_title_translated.pkl'
corpus = pd.read_pickle(fp)

In [6]:
def filter_null(df, verbose=True):
    '''filter null translations'''
    clean_corp = df[(df != ' ') & (df != '') & (df != 'source language unavailable for translation')]  
    if verbose:
        print(f'{len(df) - len(clean_corp)} null records dropped out of {len(df)}')  
    return clean_corp

In [8]:
fp = f'{directory}/data/processed/desc_title_translated.pkl'
corpus = pd.read_pickle(fp)
  
order = corpus.index
new_doc = filter_null(corpus['title_en']).reset_index()
new_doc
# docs = new_doc['title_en']
# docs
new_doc['index']

26 null records dropped out of 3895


0          0
1          1
2          2
3          3
4          4
        ... 
3864    3890
3865    3891
3866    3892
3867    3893
3868    3894
Name: index, Length: 3869, dtype: int64

In [ ]:
# ----------- topic modeling results --------------
# X [1]loop through search terms
# X [2]loop through similarity query threshold
# [3]loop through topics that are above threshold
# [4]find threshold cut offs of relevant videos from video-topics matix 
# [5]if video above threshold mark 1, else mark 0

# ----------- human annotation results --------------
# [1]loop through all available topics
# [2]find threshold cut offs of relevant videos from video-topics matix  (0.5) lower the better check n of video total to review
# [3][manual] loop through videos, show video, record human annonatation of hunting activity present (1) or not (0)


# threshold vs precision + recal curve
# precison = TP/(TP+FP)     
# recall = TP/(TP+FN)

In [ ]:
# loop through all available topics
# find threshold cut offs of relevant videos from video-topics matix  (0.5) lower the better check n of video total to review

{
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
...
 }

# grab video per topic > 0.5 for human labeling


